In [26]:
import os
import sys
import math
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import matplotlib.pyplot as plt
plt.rc('font', family = 'AppleGothic')
plt.rc('axes', unicode_minus = False)
%matplotlib inline

In [3]:
data_v3 = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', '5th_FE_ver3.pkl'))
data_v4 = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', '5th_FE_ver4.pkl'))

In [4]:
import xgboost as xgb
def MAPE(y, y_pred):
    return np.mean(np.abs((y - y_pred) / y)) * 100

In [7]:
from sklearn.model_selection import train_test_split

In [9]:
xgb_pars = {'min_child_weight': 1, 'eta': 0.03, 'colsample_bytree': 0.9, 'n_estimators' : 3000,
            'max_depth': 20, 'subsample': 0.9, 'lambda': 1. , 'booster' : 'gbtree', 'silent': 1,
            'objective': 'reg:linear', 'gamma' : 0.01}

In [10]:
for i, data in enumerate([data_v3, data_v4], 3):
    locals().update(data)
    print(f'------------version_{i}--------------')
    for i in range(5):
        num = random.randint(1, 100)
        x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = num)

        dtrain = xgb.DMatrix(x_train, label = np.log(y_train))
        dtest = xgb.DMatrix(x_test)

        watchlist = [(dtrain, 'train')]
        model = xgb.train(xgb_pars, dtrain, 10000, watchlist, early_stopping_rounds = 300, verbose_eval = False)
        y_pred = model.predict(dtest)

        value = MAPE(y_test, np.exp(y_pred))
        print(f'{i}_MAPE: ', value)

------------version_3--------------
0_MAPE:  32.45844420995544
1_MAPE:  32.36140101818138
2_MAPE:  32.52659564356228
3_MAPE:  32.25996445585205
4_MAPE:  33.82853123018747
------------version_4--------------
0_MAPE:  34.113742950074325
1_MAPE:  33.47732233591195
2_MAPE:  32.18399951428792


KeyboardInterrupt: 

In [ ]:
# 판매량 예측 후 취급액 예측

In [ ]:
new_y = (y / X['판매단가'])

In [41]:
for i, data in enumerate([data_v3, data_v4], 3):
    locals().update(data)
    print(f'------------version_{i}--------------')
    for i in range(5):
        num = random.randint(1, 100)
        x_train, x_test, y_train, y_test = train_test_split(X, new_y, random_state = num)
        dtrain = xgb.DMatrix(x_train, label = y_train)
        dtest = xgb.DMatrix(x_test)

        watchlist = [(dtrain, 'train')]
        model = xgb.train(xgb_pars, dtrain, 10000, watchlist, early_stopping_rounds = 300, verbose_eval = False)
        y_pred = model.predict(dtest)
        
        y_true = y.loc[y_test.index]
        y_pred = x_test['판매단가'] * y_pred
        
        value = MAPE(y_true, y_pred)
        print(f'{i}_MAPE: ', value)

------------version_3--------------
0_MAPE:  47.041303043854754
1_MAPE:  44.636843911179156


KeyboardInterrupt: 